## Get info by pdb_id and search info for lysozyme

In [1]:
import pypdb
import nglview as nv
import MDAnalysis as mda
import Bio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from urllib.parse import quote
import urllib


In [8]:
# Написать функцию, которая будет загружать файл из RCSB и анализировать его свойства при помощи Mdanalysis, DSSP, freesasa (Бонусное задание Propka). Минимальный набор свойств: Радиус гирации, индекс гидрофобности, доступная растворителю площадь, pI, атомная масса. Минимальная информация о белке Uniprot ID, организм, из которого получен белок.
# Найти в RCSB все белки лизоцима(lysozyme) с разрешением 1,5 и лучше.
# Применить к найденым белкам функцию, результаты представить в виде Pandas Dataframe и сохранить в CSV файл.


In [16]:
def get_info(pdb_id):

    info = {}

    try:
        !cd pdbs
        from Bio.PDB import PDBParser
        from Bio.PDB.DSSP import DSSP
        from Bio.SeqUtils.ProtParam import ProteinAnalysis,ProtParamData
        from Bio.PDB.Polypeptide import three_to_one, one_to_three
        import freesasa

        file_name = pdb_id+'.pdb'

        file_ = open(file_name,'w')
        file_.write(pypdb.get_pdb_file(pdb_id))

        # all_info = pypdb.get_info(pdb_id)
        entity = pypdb.describe_pdb(pdb_id+'/1',url_root='https://data.rcsb.org/rest/v1/core/polymer_entity/')


        info["pdb_id"] = pdb_id
        info["file_name"] = file_name
        info["organism"] = entity["rcsb_entity_source_organism"][0]["ncbi_scientific_name"]
        info["uniprot_ids"] = ','.join(entity["rcsb_polymer_entity_container_identifiers"]["uniprot_ids"])



        molecule=mda.Universe(file_name)

        seq_3_base=molecule.select_atoms('protein').residues
        aaa_dict = "Ala,Arg,Asn,Asp,Cys,Gln,Glu,Gly,His,Hyl,Hyp,Ile,Leu,Lys,Met,Phe,Pro,Ser,Thr,Trp,Tyr,Val".upper().split(",")
        sequence = []
        for aa in seq_3_base:
            if aa.resname in aaa_dict:
                sequence.append(three_to_one(aa.resname))
            # else:
            #     sequence.append('X')
        sequence = ''.join(sequence)

        #оценка свойств белка
        pa=ProteinAnalysis(sequence)


        info["sequence"] = sequence
        info["standart_acids_percent"] = len(sequence) / len(seq_3_base)

        selection = molecule.select_atoms('protein')
        info["radius_of_gyration"] =selection.radius_of_gyration()
        info["atomic_mass"] = selection.total_mass()
        info["center_of_mass"] = selection.center_of_mass()
        info["center_of_geometry"] = selection.center_of_geometry()

        #процент ароматических кислот
        info["aromaticity"] = pa.aromaticity()
        #заряд при заданном pH
        info["charge_at_pH_7"] = pa.charge_at_pH(7.0)
        #изолектрическая точка - тот pH при котором заряд 0
        info["isoelectric_point"] = pa.isoelectric_point()
        #индекс нестабильности белка
        info["instability_index"] = pa.instability_index()
        #кофф мол экстинция по числу аром кислот, поглощающих ультрафиолет
        info["molar_extinction_coefficient"] = pa.molar_extinction_coefficient()
        #процент содержания различных 2 стр
        str_2 = pa.secondary_structure_fraction() # helix, turn, sheet
        info["helix_percent"] = str_2[0] 
        info["turn_percent"] = str_2[1] 
        info["sheet_percent"] = str_2[2] 
        #скор гидрофобности белка (+ - фобный, - фильный)
        info["gravy"] = pa.gravy()
        # info["acids_percent"] = pa.get_amino_acids_percent()


        #расчёт площади доступности
        p = PDBParser()
        structure = p.get_structure(pdb_id, file_name)
        model = structure[0]

        dssp = DSSP(model, file_name)

        columns='dssp index,amino acid,secondary structure,relative ASA,phi,psi,NH_O_1_relidx,NH_O_1_energy,O_NH_1_relidx,O_NH_1_energy,NH_O_2_relidx,NH_O_2_energy,O_NH_2_relidx,O_NH_2_energy'.split(',')
        df=pd.DataFrame(dssp.property_list,columns=columns)
        df['Kyte-Doolitle']=df['amino acid'].map(ProtParamData.kd)


        freesasa_structure = freesasa.Structure(file_name)
        result = freesasa.calc(freesasa_structure)
        area_classes = freesasa.classifyResults(result, freesasa_structure)
        resids=[aa.resnum for aa in molecule.select_atoms('protein').residues]
        selections=[ f'r_{resid}, resi {resid}' for resid in resids]
        sasa=freesasa.selectArea(selections, freesasa_structure, result)
        df['sasa']=sasa.values()

        info['freesasa']=(df['Kyte-Doolitle']*(df['sasa']/result.totalArea())).sum()
    

    except Exception as error:
        info["error"] = error

    return info


In [57]:
querry={
  "query": {
    "type": "group",
    "logical_operator": "and",
    "nodes": [
      {
        "type": "terminal",
        "service": "text",
        "parameters": {
          "value": "lysozyme"
        }
      },
      {
        "type": "terminal",
        "service": "text",
        "parameters": {
          "operator": "exact_match",
          "value": "Protein (only)",
          "attribute": "rcsb_entry_info.selected_polymer_entity_types"
        }
      },
      {
        "type": "terminal",
        "service": "text",
        "parameters": {
          "operator": "less_or_equal",
          "value": 1.5,
          "attribute": "rcsb_entry_info.resolution_combined"
        }
      },
    
    ]
  },
  "return_type": "entry",
  "request_options": {
    "pager": {
      "start": 0,
      "rows": 1000
    },
    "scoring_strategy": "combined",
    "sort": [
      {
        "sort_by": "score",
        "direction": "desc"
      }
    ]
  }
}
url_root = 'https://search.rcsb.org/rcsbsearch/v1/query?json='

response=urllib.request.urlopen(url_root+quote(json.dumps(querry)))
results=json.load(response)

pdb_ids = [entity_["identifier"] for entity_ in results['result_set']]
len(pdb_ids)


760

In [61]:
import pandas as pd 
import warnings

info_df = pd.DataFrame(columns=['pdb_id'])

warnings.filterwarnings("ignore")

for i in range(len(pdb_ids)):
    print(str(i)+'/'+str(len(pdb_ids))+' '+pdb_ids[i])
    result = get_info(pdb_ids[i])
    info_df = info_df.append(result, ignore_index=True)

info_df.to_csv('final_table.csv')

0/760 1LKS
1/760 1HF4
2/760 4G9S
3/760 7JMU
4/760 1REX
5/760 5V92
6/760 6A10
7/760 2NWD
8/760 6D9I
9/760 1V7T
10/760 6SYC
11/760 7DER
12/760 1WTN
13/760 5V8G
14/760 135L
15/760 3QE8
16/760 2FBB
17/760 6TVL
18/760 2D4I
19/760 2D4K
20/760 1IEE
21/760 1GBS
22/760 3WL2
23/760 2Z2F
24/760 2IHL
25/760 4LZT
26/760 3LZT
27/760 3AGI
28/760 3AGH
29/760 3GUN
30/760 3GUP
31/760 5V4H
32/760 5V4G
33/760 5V4I
34/760 193L
35/760 194L
36/760 1JSE
37/760 1LZ1
38/760 6F1R
39/760 6F1L
40/760 6F1M
41/760 6F1P
42/760 6F1O
43/760 6F9Z
44/760 6F9Y
45/760 6F9X
46/760 6FA0
47/760 6S7N
48/760 5F14
49/760 5F16
50/760 7AVE
51/760 7AVG
52/760 7AVF
53/760 3OD9
54/760 4NHI
55/760 3WMK
56/760 3N9A
57/760 3N9C
58/760 3N9E
59/760 2ZYP
60/760 2F2Q
61/760 1P7S
62/760 1VDQ
63/760 1LW9
64/760 5K2N
65/760 5K2Q
66/760 5K2P
67/760 4I7M
68/760 4NGW
69/760 3RU5
70/760 3AJN
71/760 4B4E
72/760 4B49
73/760 4B4I
74/760 4B4J
75/760 2HU3
76/760 6SYE
77/760 5HMJ
78/760 6GNL
79/760 5LXW
80/760 1ZVH
81/760 3HH3
82/760 3HH4
83/760 3HH5
84

,pdb_id,aromaticity,atomic_mass,center_of_geometry,center_of_mass,charge_at_pH_7,file_name,freesasa,gravy,helix_percent,...,isoelectric_point,molar_extinction_coefficient,organism,radius_of_gyration,sequence,sheet_percent,standart_acids_percent,turn_percent,uniprot_ids,error
0,1LKS,0.093023,13526.588,"[-16.869514808624512, -35.73533335051828, -12....","[-16.846071791747146, -35.733155217083755, -12...",7.772433,1LKS.pdb,-2.067617,-0.472093,0.248062,...,9.323756,"(37470, 37970)",Gallus gallus,13.990574,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,0.186047,1.0,0.294574,P00698,NaN
1,1HF4,0.093023,27153.303,"[6.149600393711701, 31.89072903945391, 38.7825...","[6.15786107829386, 31.886049188638363, 38.7678...",15.784765,1HF4.pdb,NaN,-0.472093,0.248062,...,9.393060,"(74940, 75940)",Gallus gallus,21.351693,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,0.186047,1.0,0.294574,P00698,Length of values (129) does not match length o...
2,4G9S,0.090604,35941.147,"[-11.844082083989473, 55.537415571879514, -8.2...","[-11.947145585578026, 55.41838737551399, -8.25...",7.351244,4G9S.pdb,NaN,-0.803020,0.268456,...,9.120745,"(45840, 45840)",Salmo salar,19.599041,HHHHHHHMDITKVDTSGASEITARQDKLTLQGVDASHKLAEHDLVR...,0.177852,1.0,0.231544,A6PZ97,Length of values (187) does not match length o...
3,7JMU,0.093023,29506.495,"[-7.114628903331944, -7.7917081455301265, -37....","[-7.003883960207267, -7.5215738938743675, -37....",15.784765,7JMU.pdb,NaN,-0.472093,0.248062,...,9.393060,"(74940, 75940)",Gallus gallus,21.152891,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,0.186047,1.0,0.294574,P00698,Length of values (129) does not match length o...
4,1REX,0.100000,13700.777,"[13.321222546518545, 14.777592818445523, 28.33...","[13.301629323947633, 14.788248470345678, 28.31...",7.774336,1REX.pdb,-2.115515,-0.485385,0.269231,...,9.277790,"(36440, 36940)",Homo sapiens,14.093618,KVFERCELARTLKRLGMDGYRGISLANWMCLAKWESGYNTRATNYN...,0.207692,1.0,0.223077,P61626,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,3CBP,0.111111,25778.118,"[28.20835405677741, 3.750552413816193, 15.8674...","[28.275105681367396, 3.768771257647115, 15.879...",-16.536053,3CBP.pdb,NaN,-0.376955,0.300412,...,4.795756,"(38850, 39100)",Homo sapiens,18.871339,GVCWIYYPDGGSLVGEVNEDGEMTGEKIAYVYPDERTALYGKFIDG...,0.255144,1.0,0.251029,Q8WTS6,Length of values (239) does not match length o...
756,3MRG,0.120104,45495.092,"[3.4391476688653415, -0.27462587713908276, 9.8...","[3.477968472142521, -0.2951673751614714, 9.864...",-6.765865,3MRG.pdb,NaN,-0.754830,0.276762,...,6.111920,"(101300, 101800)",Homo sapiens,22.890894,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,0.221932,1.0,0.185379,P04439,Length of values (275) does not match length o...
757,6NB9,0.071429,1372.567,"[3.730831636781139, 3.8100204142380734, 7.8873...","[3.566781258970837, 3.8687577621395657, 8.3312...",-0.238087,6NB9.pdb,NaN,0.750000,0.357143,...,6.001425,"(0, 0)",Homo sapiens,8.372868,FAEVGSNKGAIIGL,0.285714,1.0,0.357143,P05067,Length of values (14) does not match length of...
758,6R7W,0.102113,30559.238,"[-11.104867835309275, 1.5737865140723348, -20....","[-11.140130752620854, 1.5757023530442957, -20....",-5.448555,6R7W.pdb,-1.426491,-0.591197,0.260563,...,5.635950,"(38390, 38640)",Tannerella forsythia,18.478448,PSSTILIPVVVHVVYNNSAQNISDAQIISQIQVLNEDFRRMNADQA...,0.179577,1.0,0.285211,G8ULV1,NaN


In [63]:
info_df

,pdb_id,aromaticity,atomic_mass,center_of_geometry,center_of_mass,charge_at_pH_7,file_name,freesasa,gravy,helix_percent,...,isoelectric_point,molar_extinction_coefficient,organism,radius_of_gyration,sequence,sheet_percent,standart_acids_percent,turn_percent,uniprot_ids,error
0,1LKS,0.093023,13526.588,"[-16.869514808624512, -35.73533335051828, -12....","[-16.846071791747146, -35.733155217083755, -12...",7.772433,1LKS.pdb,-2.067617,-0.472093,0.248062,...,9.323756,"(37470, 37970)",Gallus gallus,13.990574,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,0.186047,1.0,0.294574,P00698,NaN
1,1HF4,0.093023,27153.303,"[6.149600393711701, 31.89072903945391, 38.7825...","[6.15786107829386, 31.886049188638363, 38.7678...",15.784765,1HF4.pdb,NaN,-0.472093,0.248062,...,9.393060,"(74940, 75940)",Gallus gallus,21.351693,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,0.186047,1.0,0.294574,P00698,Length of values (129) does not match length o...
2,4G9S,0.090604,35941.147,"[-11.844082083989473, 55.537415571879514, -8.2...","[-11.947145585578026, 55.41838737551399, -8.25...",7.351244,4G9S.pdb,NaN,-0.803020,0.268456,...,9.120745,"(45840, 45840)",Salmo salar,19.599041,HHHHHHHMDITKVDTSGASEITARQDKLTLQGVDASHKLAEHDLVR...,0.177852,1.0,0.231544,A6PZ97,Length of values (187) does not match length o...
3,7JMU,0.093023,29506.495,"[-7.114628903331944, -7.7917081455301265, -37....","[-7.003883960207267, -7.5215738938743675, -37....",15.784765,7JMU.pdb,NaN,-0.472093,0.248062,...,9.393060,"(74940, 75940)",Gallus gallus,21.152891,KVFGRCELAAAMKRHGLDNYRGYSLGNWVCAAKFESNFNTQATNRN...,0.186047,1.0,0.294574,P00698,Length of values (129) does not match length o...
4,1REX,0.100000,13700.777,"[13.321222546518545, 14.777592818445523, 28.33...","[13.301629323947633, 14.788248470345678, 28.31...",7.774336,1REX.pdb,-2.115515,-0.485385,0.269231,...,9.277790,"(36440, 36940)",Homo sapiens,14.093618,KVFERCELARTLKRLGMDGYRGISLANWMCLAKWESGYNTRATNYN...,0.207692,1.0,0.223077,P61626,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,3CBP,0.111111,25778.118,"[28.20835405677741, 3.750552413816193, 15.8674...","[28.275105681367396, 3.768771257647115, 15.879...",-16.536053,3CBP.pdb,NaN,-0.376955,0.300412,...,4.795756,"(38850, 39100)",Homo sapiens,18.871339,GVCWIYYPDGGSLVGEVNEDGEMTGEKIAYVYPDERTALYGKFIDG...,0.255144,1.0,0.251029,Q8WTS6,Length of values (239) does not match length o...
756,3MRG,0.120104,45495.092,"[3.4391476688653415, -0.27462587713908276, 9.8...","[3.477968472142521, -0.2951673751614714, 9.864...",-6.765865,3MRG.pdb,NaN,-0.754830,0.276762,...,6.111920,"(101300, 101800)",Homo sapiens,22.890894,GSHSMRYFFTSVSRPGRGEPRFIAVGYVDDTQFVRFDSDAASQRME...,0.221932,1.0,0.185379,P04439,Length of values (275) does not match length o...
757,6NB9,0.071429,1372.567,"[3.730831636781139, 3.8100204142380734, 7.8873...","[3.566781258970837, 3.8687577621395657, 8.3312...",-0.238087,6NB9.pdb,NaN,0.750000,0.357143,...,6.001425,"(0, 0)",Homo sapiens,8.372868,FAEVGSNKGAIIGL,0.285714,1.0,0.357143,P05067,Length of values (14) does not match length of...
758,6R7W,0.102113,30559.238,"[-11.104867835309275, 1.5737865140723348, -20....","[-11.140130752620854, 1.5757023530442957, -20....",-5.448555,6R7W.pdb,-1.426491,-0.591197,0.260563,...,5.635950,"(38390, 38640)",Tannerella forsythia,18.478448,PSSTILIPVVVHVVYNNSAQNISDAQIISQIQVLNEDFRRMNADQA...,0.179577,1.0,0.285211,G8ULV1,NaN


## Propka parser

In [217]:
def parse_pka(pka_file_name):

    pka = open(pka_file_name,'r').read()
    # print(pka)


    new_block2 =  '--------------------------------------------------------------------------------------------------------\n'
    new_block1 = '-------------------------------------------------------------------------------\n'

    split1 = pka.split(new_block2)[0]
    block1 = split1.split(new_block1)[4]

    #block1 
    block1 = block1.split('\n')[8:]
    block1_info_list = []

    for row in block1:
        if len(row) != 0:
            block1_info_row = {
                "RESIDUE": {
                    "AA": row[0:3].strip(),
                    "SIZE": row[4:7].strip(), 
                    "A": row[8:9].strip()
                    },
                "pKa": row[9:16].strip(),
                "BURIED": row[17:22].strip(),
                "%": row[23:24].strip(),
                "DESOLVATION_EFFECTS": {
                    "REGULAR_1": row[25:32].strip(),
                    "REGULAR_2": row[33:37].strip(),
                    "RE_1": row[38:44].strip(),
                    "RE_2": row[45:49].strip()
                },
                "SIDECHAIN": {
                    "HYDROGEN_1": row[50:57].strip(),
                    "HYDROGEN_2": row[58:61].strip(),
                    "BOND_1": row[62:65].strip(),
                    "BOND_2": row[66:67].strip()
                },
                "BACKBONE":{
                    "HYDROGEN_1": row[68:75].strip(),
                    "HYDROGEN_2": row[76:79].strip(),
                    "BOND_1": row[80:83].strip(),
                    "BOND_2": row[84:85].strip()
                },
                "COULOMBIC_INTERACTION":{
                    "1_param": row[86:93].strip(),
                    "2_param": row[94:97].strip(),
                    "3_param": row[98:101].strip(), 
                    "4_param": row[102:103].strip()
                }
            }
            
            block1_info_list.append(block1_info_row)



    #block2
    block2 = pka.split(new_block2)[1]
    block2 = block2.split('\n')[2:]

    block2_info_list = []

    for row in block2:
        if len(row) != 0:
            row = row[3:]
            block2_info_row = {
            "Group": {
                "AA": row[0:3].strip(), 
                "SIZE": row[4:7].strip(),
                "A": row[8:9].strip()
            },
            "pKa": row[10:18].strip(),
            "model-pKa": row[19:20].strip(),
            "ligand": row[21:29].strip(),
            "atom-type": row[30:].strip()
            }
                    
            block2_info_list.append(block2_info_row)


    #block3
    block3 = pka.split(new_block2)[3]
    split2 = block3.split("\n\n")

    block31 = split2[0]
    block31 = block31.split("\n")
    block31_info_list = []
    block31 = block31[1:]
    for row in block31:
        if len(row) != 0:
            block31_info_row = {
            "pH": row[:6].strip(),
            "energy": row[7:].strip()
            }     
            block31_info_list.append(block31_info_row)


    block32_info_list = split2[1]


    block33 = split2[2]
    block33 = block33.split("\n")
    block34 = block33[-2:]
    block33 = block33[2:-2]

    block33_info_list = []

    for row in block33:
        if len(row) != 0:
            block33_info_row = {
            "pH": row[:6].strip(),
            "charge_unfolded": row[7:16].strip(),
            "charge_folded": row[17:].strip()
            }     
            block33_info_list.append(block33_info_row)


    block34 = block34[0].split(' ')
    block34_info_list = {
        'pI_folded': block34[3],
        'pI_unfolded': block34[6] 
    }


    pka_parsed = {
        "block1": block1_info_list,
        "block2": block2_info_list,
        "block3": {
            "free_energy vs pH": block31_info_list,
            "other_stuff": block32_info_list, 
            "charge vs pH": block33_info_list,
            "pI": block34_info_list
        }
    }


    return pka_parsed

In [224]:

#может не работать адекватно из кода, пытается вызвать propka3 не из виртуального окружения, хотя оно активно
# !Set-ExecutionPolicy -ExecutionPolicy Unrestricted
# !conda init powershell
# !conda activate struct_bioinf && propka3 file_name

import os

!cd pdbs
#для пробки нужно только белок
pdb_id = pdb_ids[1]
# pdb_id = '1HF4'

file_name = pdb_id+'.pdb'
molecule=mda.Universe(file_name)

file_name = pdb_id+'_prot.pdb'
protein=molecule.select_atoms('protein')
protein.write(file_name)
file_name

pka_file_name = pdb_id+'_prot.pka'
os.system('conda activate struct_bioinf && propka3 "'+file_name+'"')

print(pka_file_name)
parsed_pka = parse_pka(pka_file_name)
print(parsed_pka.keys())
print(parsed_pka["block3"].keys())
parsed_pka

1HF4_prot.pka
dict_keys(['block1', 'block2', 'block3'])
dict_keys(['free_energy vs pH', 'other_stuff', 'charge vs pH', 'pI'])


ASP',
    '3_param': '48',
    '4_param': 'B'}},
  {'RESIDUE': {'AA': 'ARG', 'SIZE': '45', 'A': 'B'},
   'pKa': '',
   'BURIED': '',
   '%': '',
   'DESOLVATION_EFFECTS': {'REGULAR_1': '',
    'REGULAR_2': '',
    'RE_1': '',
    'RE_2': ''},
   'SIDECHAIN': {'HYDROGEN_1': '0.00',
    'HYDROGEN_2': 'XXX',
    'BOND_1': '0',
    'BOND_2': 'X'},
   'BACKBONE': {'HYDROGEN_1': '0.00',
    'HYDROGEN_2': 'XXX',
    'BOND_1': '0',
    'BOND_2': 'X'},
   'COULOMBIC_INTERACTION': {'1_param': '0.06',
    '2_param': 'TYR',
    '3_param': '53',
    '4_param': 'B'}},
  {'RESIDUE': {'AA': 'ARG', 'SIZE': '45', 'A': 'B'},
   'pKa': '',
   'BURIED': '',
   '%': '',
   'DESOLVATION_EFFECTS': {'REGULAR_1': '',
    'REGULAR_2': '',
    'RE_1': '',
    'RE_2': ''},
   'SIDECHAIN': {'HYDROGEN_1': '0.00',
    'HYDROGEN_2': 'XXX',
    'BOND_1': '0',
    'BOND_2': 'X'},
   'BACKBONE': {'HYDROGEN_1': '0.00',
    'HYDROGEN_2': 'XXX',
    'BOND_1': '0',
    'BOND_2': 'X'},
   'COULOMBIC_INTERACTION': {'1_param': 

## Домашнее задание:
* Написать функцию, которая будет загружать файл из RCSB и анализировать его свойства при помощи Mdanalysis, DSSP, freesasa (Бонусное задание Propka). Минимальный набор свойств: Радиус гирации, индекс гидрофобности, доступная растворителю площадь, pI, атомная масса. Минимальная информация о белке Uniprot ID, организм, из которого получен белок.
* Найти в RCSB все белки лизоцима(lysozyme) с разрешением 1,5 и лучше. 
* Применить к найденым белкам функцию, результаты представить в виде Pandas Dataframe и сохранить в CSV файл.

### Бонусные задания:
* написать парсер результатов работы программы Propka
* функция обработчик должна представлять результаты обработки с разбиением по цепям. (понимает, одна или несколько цепей и разбивает по ним)